In [2]:
import pickle
import os
import numpy as np

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
folder = os.path.abspath(r'./variables')

In [ ]:
filename = open(folder + r'/musician_sequential_0.pkl', 'rb')
sample0 = pickle.load(filename)

filename = open(folder + r'/musician_sequential_1.pkl', 'rb')
sample1 = pickle.load(filename)

In [14]:
influencer_feature = np.concatenate([sample0[0], sample1[0]])
follower_feature = np.concatenate([sample0[1], sample1[1]])

with_influence = len(sample1[0])
without_influence = len(sample0[0])

target = np.array([1 for _ in range(with_influence)] + [0 for _ in range(without_influence)])

In [16]:
idx = np.array(range(with_influence + without_influence))
np.random.shuffle(idx)

influencer_feature = influencer_feature[idx, :, :]
follower_feature = follower_feature[idx, :, :]
target = target[idx]

In [20]:
target.shape

(91145,)

In [26]:
# 按照7: 1.5: 1.5划分训练集、验证集、测试集
train_influencer = influencer_feature[:63801, :, :]
train_follower = follower_feature[:63801, :, :]
train_y = target[:63801]

val_influencer =  influencer_feature[63801:77473, :, :]
val_follower = follower_feature[63801:77473, :, :]
val_y = target[63801:77473]

test_influencer =  influencer_feature[77473:, :, :]
test_follower = follower_feature[77473:, :, :]
test_y = target[77473:]

In [27]:
from keras import layers
import keras
from keras import Input
from keras.models import Sequential, Model

In [11]:
monitor = [
    keras.callbacks.EarlyStopping(
        monitor = 'acc',
        patience = 3,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath = 'influence_musicians.h5',
        monitor = 'val_loss',
        save_best_only = True,
    )
]

In [12]:
# RNN 结构
influencer_input = Input(shape = (20, 33), dtype = "float64", name = "influencer")
memory_influencer = layers.LSTM(512, input_shape = (20, 33), dropout = 0.3)(influencer_input)
memory_influencer = layers.Dense(512, activation = "relu")(memory_influencer)
memory_influencer = layers.Dropout(0.5)(memory_influencer)

follower_input = Input(shape = (20, 33), dtype = "float64", name = "follower") 
memory_follower = layers.LSTM(256, input_shape = (20, 33), dropout = 0.1)(follower_input)
memory_follower = layers.Dense(128, activation = "relu")(memory_follower)
memory_influencer = layers.Dropout(0.5)(memory_influencer)

concatenated = layers.concatenate([memory_influencer, memory_follower], axis = -1)

existance = layers.Dense(256, activation = "relu")(concatenated)
existance = layers.Dense(128, activation = "relu")(existance)
existance = layers.Dense(64, activation = "relu")(existance)
existance = layers.Dense(1, activation = "sigmoid")(existance)

model = Model([influencer_input, follower_input], existance)
optimizer = keras.optimizers.RMSprop(lr = 0.01)
model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics = ["acc"])

2023-10-28 19:57:31.553252: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-28 19:57:31.555769: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-28 19:57:31.558589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [13]:
history = model.fit([train_influencer, train_follower], train_y, 
          batch_size = 256, epochs = 100, 
          validation_data = ([val_influencer, val_follower], val_y), callbacks = monitor)

Epoch 1/100


2023-10-28 19:57:33.047732: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-28 19:57:33.050625: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-28 19:57:33.056526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

223/223 [==============================] - ETA: 0s - loss: 0.8743 - acc: 0.5317

2023-10-28 19:58:02.042612: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-28 19:58:02.045013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-28 19:58:02.046914: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

223/223 [==============================] - 32s 118ms/step - loss: 0.8743 - acc: 0.5317 - val_loss: 0.6927 - val_acc: 0.5072
Epoch 2/100
223/223 [==============================] - 24s 110ms/step - loss: 0.6817 - acc: 0.5311 - val_loss: 0.6587 - val_acc: 0.5579
Epoch 3/100
223/223 [==============================] - 26s 116ms/step - loss: 0.6689 - acc: 0.5475 - val_loss: 0.6577 - val_acc: 0.5578
Epoch 4/100
223/223 [==============================] - 26s 117ms/step - loss: 0.6634 - acc: 0.6089 - val_loss: 0.6107 - val_acc: 0.6560
Epoch 5/100
223/223 [==============================] - 26s 118ms/step - loss: 0.5934 - acc: 0.6844 - val_loss: 0.5443 - val_acc: 0.7144
Epoch 6/100
223/223 [==============================] - 26s 116ms/step - loss: 0.5523 - acc: 0.7254 - val_loss: 0.5615 - val_acc: 0.7288
Epoch 7/100
223/223 [==============================] - 26s 118ms/step - loss: 0.5283 - acc: 0.7453 - val_loss: 0.4927 - val_acc: 0.7700
Epoch 8/100
223/223 [==============================] - 26s 1

In [42]:
max(history.history["val_acc"])

0.8539583683013916

In [2]:
from keras.models import load_model

2023-10-29 14:31:56.335062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 14:31:58.149079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
influence_musicians = load_model("influence_musicians8570.h5")

2023-10-29 14:32:01.609382: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-29 14:32:01.797491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-29 14:32:01.798147: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-29 14:32:01.800741: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-29 14:32:01.801453: I tensorflow/compile

In [20]:
influence_musicians.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 influencer (InputLayer)        [(None, 20, 33)]     0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 512)          1118208     ['influencer[0][0]']             
                                                                                                  
 follower (InputLayer)          [(None, 20, 33)]     0           []                               
                                                                                                  
 dense (Dense)                  (None, 512)          262656      ['lstm[0][0]']                   
                                                                                              

In [ ]:
# 利用验证集找到benchmark来判断variable是0还是1

val_prob = influence_musicians.predict([val_influencer, val_follower])

hash_table = {}

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
for bar in np.linspace(0.35, 0.45, 1000):
    val_pred = np.array([1 if val_prob[i] >= bar else 0 for i in range(len(val_prob))])
    accuracy = accuracy_score(val_y, val_pred)
    hash_table[bar] = accuracy

In [36]:
list(hash_table.keys())[np.argmax(np.array(list(hash_table.values())))]

0.40895895895895895

In [37]:
bar = 0.40896

In [38]:
test_prob = influence_musicians.predict([test_influencer, test_follower])

382/382 [==============================] - 5s 12ms/step


In [39]:
test_pred = np.array([1 if test_prob[i] >= bar else 0 for i in range(len(test_prob))])

In [41]:
accuracy = accuracy_score(test_y, test_pred)
accuracy

0.8423338523313939

In [44]:
from sklearn.metrics import roc_curve, auc

In [46]:
fpr, tpr, _  = roc_curve(test_y, test_pred)
auc(fpr, tpr)

0.8424902409058185